In [1]:
JIDT_HOME='/home/jovyan/notebooks/jidt'
DATA_FILENAME=JIDT_HOME + '/demos/data/2coupledRandomCols-1.txt'

In [2]:
#---------------------------------------------------
# START JVM
#-----------------------------------------------------

from jpype import startJVM, isJVMStarted, getDefaultJVMPath, JArray, JInt, JPackage, shutdownJVM, JDouble
import numpy
import sys
# Our python data file readers are a bit of a hack, python users will do better on this:
sys.path.append("/home/jovyan/notebooks/jidt/demos/python")
import readIntsFile, readFloatsFile

# Add JIDT jar library to the path
jarLocation = "/home/jovyan/notebooks/jidt/infodynamics.jar"
# Start the JVM (add the "-Xmx" option with say 1024M if you get crashes due to not enough memory space)
if (not isJVMStarted()):
    startJVM(getDefaultJVMPath(), "-ea", "-Djava.class.path=" + jarLocation)


In [3]:
# LOAD DATA FILE
#-------------------

import pandas as pd
#from pathlib import Path

filename = DATA_FILENAME
data = pd.read_csv(filename, delim_whitespace=True, comment='%', header=None, names = ['Col0', 'Col1'], index_col=None)
data.head()

,Col0,Col1
0,0.625190,0.000000
1,0.183227,-0.847045
2,-1.029767,0.131580
3,0.949222,-0.989124
4,0.307062,0.680356


In [5]:
import matplotlib.pyplot as plt
%matplotlib widget

fig, axs = plt.subplots(1, 1, figsize=(9, 6))  
#axs.plot(data['Heart rate'], label='Heart rate')
axs.plot(data['Col0'], label='Col0')
axs.plot(data['Col1'], label='Col1')
#axs.plot(data['Blood oxygen'], label='Blood oxygen')
#axs.set_title('Heart rate, Chest Volume and Blood Oxygen vs Time')
axs.set_xlabel('Sample index')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
# Select data source and destination for MI Estimation
# ----------------------------------------------------

import numpy as np
source = data['Col0'].to_numpy()
destination = data['Col1'].to_numpy()


In [75]:

# # 0. Load/prepare the data:
# dataRaw = readFloatsFile.readFloatsFile(JIDT_HOME + "/demos/data/SFI-heartRate_breathVol_bloodOx-extract.txt")
# # As numpy array:
# data = numpy.array(dataRaw)
# source = data[:,0]
# destination = data[:,1]

In [7]:
# Conversion to Java double array instead of numpy.float
source = JArray(JDouble, 1)(source.tolist())
destination = JArray(JDouble, 1)(destination.tolist())

In [8]:
# Construct the calculator : MI Kernel Estimator
#-------------------------------------------------

# 1. Construct the calculator:
calcClass = JPackage("infodynamics.measures.continuous.kernel").MutualInfoCalculatorMultiVariateKernel
calc = calcClass()
# 2. Set any properties to non-default values:
calc.setProperty("TIME_DIFF", "0")
calc.setProperty("KERNEL_WIDTH", "0.25")
# 3. Initialise the calculator for (re-)use:
calc.initialise()
# 4. Supply the sample data:
calc.setObservations(source, destination)
# 5. Compute the estimate:
result = calc.computeAverageLocalOfObservations()

print("MI_Kernel(col_0 -> col_1) = %.4f bits" %
    (result))


MI_Kernel(col_0 -> col_1) = 0.4779 bits


In [11]:

def kernelEstimator(timediff: int, k: float):
    calc.setProperty("TIME_DIFF", str(timediff))
    calc.setProperty("KERNEL_WIDTH", str(k))
    # 3. Initialise the calculator for (re-)use:
    calc.initialise()
    # 4. Supply the sample data:
    calc.setObservations(source, destination)
    # 5. Compute the estimate:
    result = calc.computeAverageLocalOfObservations()
    return result




    

In [13]:
ks = np.linspace(0.1, 0.9, 20)
tdiffs = range(50)

KE_Results = np.zeros((len(tdiffs), ks.shape[0]))

for i, tdiff in enumerate(tdiffs):
    for j, k in enumerate(ks):
        KE_Results[i][j]=kernelEstimator(str(tdiff), str(k))


In [19]:
fig, axs = plt.subplots(1, 1, figsize=(9, 6))  
axs.plot(KE_Results[:,0], label='MI Kernel r={}'.format(ks[0]))
axs.plot(KE_Results[:,5], label='MI Kernel r={}'.format(ks[5]))
axs.plot(KE_Results[:,10], label='MI Kernel r={}'.format(ks[10]))
axs.plot(KE_Results[:,19], label='MI Kernel r={}'.format(ks[19]))
axs.set_title('MI Kernel')
axs.set_xlabel('Time Lag')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [88]:
#------------------------------------------
# Estimation with Kraskov KSG Estimator
#------------------------------------------

# 1. Construct the calculator:
calcClass = JPackage("infodynamics.measures.continuous.kraskov").MutualInfoCalculatorMultiVariateKraskov2
calc = calcClass()
# 2. Set any properties to non-default values:
calc.setProperty("k", "4")
# 3. Initialise the calculator for (re-)use:
calc.initialise()
# 4. Supply the sample data:
calc.setObservations(source, destination)
# 5. Compute the estimate:
result = calc.computeAverageLocalOfObservations()

print("MI_Kraskov (KSG) alg. 2(col_0 -> col_1) = %.4f nats" %
    (result))

MI_Kraskov (KSG) alg. 2(col_0 -> col_1) = 0.1349 nats


In [89]:
KSG_results=[]
def ksgEstimator(timediff: int):
    calc.setProperty("TIME_DIFF", str(timediff))
    # 3. Initialise the calculator for (re-)use:
    calc.initialise()
    # 4. Supply the sample data:
    calc.setObservations(source, destination)
    # 5. Compute the estimate:
    result = calc.computeAverageLocalOfObservations()
    return result

for timediff in range(50):
    result=gaussianEstimator(timediff)
    KSG_results.append(result)

In [92]:
fig, axs = plt.subplots(1, 1, figsize=(9, 6))  
axs.plot([t for t in range(50)], KSG_results, label='MI KSG 2')
axs.plot([t for t in range(50)], results, label='MI Gaussian')
axs.set_title('MI Heart rate; Chest Volume vs Time Lag')
axs.set_xlabel('Time Lag')
plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …